In [3]:
import numpy as np

# SudokuEntry Class

In [348]:
class SudokuEntry:
    def __init__(self, input_entry, size=9, indecisive=0):
        possible = np.asarray([input_entry]).reshape((-1,))
        if possible.shape[0] == 1 and possible[0] == indecisive:
                self.possible = np.ones(size, dtype=bool)
        else:
            self.possible = np.zeros(size, dtype=bool)
            self.possible[possible - 1] = 1
    
    def count(self):
        return self.possible.sum()
    
    def weightage(self):
        return self.possible * np.arange(1, self.possible.shape[0]+1)
    
    def as_set(self):
        s = set(self.weightage())
        if 0 in s: s.remove(0)
        return s
    
    def value(self):
        return 0 if self.count() > 1 else self.weightage().sum()
    
    def __str__(self):
        return str(self.value())
    
    def __repr__(self):
        return f"SudokuEntry{self.as_set()}"
    
    def __contains__(self, key):
        if not (0 < key <= self.possible.shape[0]):
            raise IndexError(f"{key} is out of bound of {repr(self)}, Range: [1, {self.possible.shape[0]}]")
        return self.possible[key-1]
    
    def __eq__(self, value):
        return self.value() == value
    
    def __add__(self, other):
        if isinstance(other, SudokuEntry):
            new = SudokuEntry(0, size=self.possible.shape[0])
            new.possible = self.possible | other.possible
            return new
        else:
            raise ValueError("Unsupported operand type for -: 'SudokuEntry' and {}".format(type(other)))
    
    
    def __sub__(self, other):
        if isinstance(other, SudokuEntry):
            new = self + other
            new.possible = new.possible ^ other.possible
            return new
        else:
            raise ValueError("Unsupported operand type for -: 'SudokuEntry' and {}".format(type(other)))
    
    def __mul__(self, scalar):
        if isinstance(scalar, (int, float)):
            new = SudokuEntry(0, size=self.possible.shape[0])
            if scalar: new.possible = self.possible
            return new
        else:
            raise ValueError("Unsupported operand type for *: 'SudokuEntry' and {}".format(type(scalar)))

    def remove_possibility(self, val):
        if val in self:
            self.possible[val-1] = False
            
    def set_val(self, vals):
        self.possible.fill(0)
        self.possible[np.asarray(vals)-1] = 1
        
    def copy(self):
        return SudokuEntry([*self.as_set()])

s = SudokuEntry([6, 7, 3])
print(s)
s

0


SudokuEntry{3, 6, 7}

# SudokuEntry Demo:

In [349]:
s.copy()

SudokuEntry{3, 6, 7}

In [530]:
s1 = SudokuEntry([1, 4])
s2 = SudokuEntry([1, 2, 3])
s1-s2

SudokuEntry{4}

In [533]:
s1*0

SudokuEntry{1, 2, 3, 4, 5, 6, 7, 8, 9}

In [392]:
s.set_val([3, 4])
s

SudokuEntry{3, 4}

In [228]:
s.remove_possibility(3)

In [224]:
s.count()

1

In [225]:
s.as_set()

{6}

In [101]:
6 in s

True

In [213]:
9 in s

9


False

In [103]:
s == 6

False

In [104]:
s == 0

True

In [106]:
0 in [s]

True

# Sudoku Class:

In [421]:
import numpy as np

class Sudoku(np.ndarray):
    def __new__(cls, input_array):
        obj = np.vectorize(lambda x: SudokuEntry(x, len(input_array) * len(input_array[0])))(np.asarray(input_array).view(cls))
        if len(obj.shape) != 4:
            raise ValueError("Input array must have shape (block_rows, block_cols, inner_rows, inner_cols)")
        return obj

    def __array_finalize__(self, obj):
        if obj is None:
            return

    def get(self, row, col):
        row_block = row // self.shape[2]
        inner_row = row % self.shape[2]
        col_block = col // self.shape[3]
        inner_col = col % self.shape[3]
        return super(Sudoku, self).__getitem__((row_block, col_block, inner_row, inner_col))
    
    def get_row(self, row, reduce=True):
        block_row = row // self.shape[2]
        inner_row = row % self.shape[2]
        ret = super(Sudoku, self).__getitem__((block_row, slice(None, None, None), inner_row, slice(None, None, None)))
        if reduce: return ret.reshape((-1,))
        return ret
        
    def rows(self, reduce=True):
        for i in range(self.shape[0]*self.shape[1]): yield self.get_row(i, reduce=reduce)
    
    def get_col(self, col, reduce=True):
        block_col = col // self.shape[3]
        inner_col = col % self.shape[3]
        ret = super(Sudoku, self).__getitem__((slice(None, None, None), block_col, slice(None, None, None), inner_col))
        if reduce: return ret.reshape((-1,))
        return ret
    
    def cols(self, reduce=True):
        for i in range(self.shape[0]*self.shape[1]): yield self.get_col(i, reduce=reduce)
            
    def get_block(self, block_row, block_col, reduce=False):
        if reduce: return self[block_row, block_col].reshape((-1,))
        else: return self[block_row, block_col]
    
    def blocks(self, reduce=False, enum=False):
        for block_row in range(self.shape[0]):
            for block_col in range(self.shape[1]):
                if enum: yield block_row, block_col, self.get_block(block_row, block_col, reduce=reduce)
                else: yield self.get_block(block_row, block_col, reduce=reduce)
        
    def __setitem__(self, item, value):
        if isinstance(item, tuple) and len(item) == 4: super().__setitem__(item, value)
        raise IndexError("Wrong indexing for Sudoku: must need 4 indices")

    def __str__(self):
        try:
            lines = []
            for block_row in range(self.shape[0]):
                for row in range(self.shape[2]):
                    line = ""
                    for block_col in range(self.shape[1]):
                        for col in range(self.shape[3]):
                            value = self[block_row, block_col, row, col]
                            line += f" {str(value)} "
                        if block_col < self.shape[1] - 1:
                            line += "|"
                    lines.append(line)
                if block_row < self.shape[0] - 1:
                    lines.append("+".join('-'*(3*self.shape[3]) for i in range(self.shape[1])))
            return "\n".join(lines)
        except:
            return super().__repr__()
    
    def __repr__(self):
        try:
            lines = []
            for block_row in range(self.shape[0]):
                for row in range(self.shape[2]):
                    line = ""
                    for block_col in range(self.shape[1]):
                        for col in range(self.shape[3]):
                            value = self[block_row, block_col, row, col]
                            line += f" {'-'.join(repr(value)[12:-1].split(', '))} "
                        if block_col < self.shape[1] - 1:
                            line += "|"
                    lines.append(line)
                if block_row < self.shape[0] - 1:
                    lines.append("+".join('-'*(3*self.shape[3]) for i in range(self.shape[1])))
            return "\n".join(lines)
        except:
            return super().__repr__()
    
    def copy(self):
        return np.vectorize(lambda x: x.copy())(super().copy())


# Example usage:
sudoku_grid = np.array([
    [[[5, 3, 0], [6, 0, 0], [0, 9, 8]],
    [[0, 7, 0], [1, 9, 5], [0, 0, 0]],
    [[0, 0, 0], [0, 0, 0], [0, 6, 0]]],
    [[[8, 0, 0], [4, 0, 0], [7, 0, 0]],
    [[0, 6, 0], [8, 0, 3], [0, 2, 0]],
    [[0, 0, 3], [0, 0, 1], [0, 0, 6]]],
    [[[0, 6, 0], [0, 0, 0], [0, 0, 0]],
    [[0, 0, 0], [4, 1, 9], [0, 8, 0]],
    [[2, 8, 0], [0, 0, 5], [6, 0, 0]]]
])

sudoku = Sudoku(sudoku_grid)
print("Sudoku grid:")
print(sudoku)
# sudoku.get_row(1)

Sudoku grid:
 5  3  0 | 0  7  0 | 0  0  0 
 6  0  0 | 1  9  5 | 0  0  0 
 0  9  8 | 0  0  0 | 0  6  0 
---------+---------+---------
 8  0  0 | 0  6  0 | 0  0  3 
 4  0  0 | 8  0  3 | 0  0  1 
 7  0  0 | 0  2  0 | 0  0  6 
---------+---------+---------
 0  6  0 | 0  0  0 | 2  8  0 
 0  0  0 | 4  1  9 | 0  0  5 
 0  0  0 | 0  8  0 | 6  0  0 


# Sudoku Class Demo:

In [404]:
sudoku

 5  3  1-2-3-4-5-6-7-8-9 | 1-2-3-4-5-6-7-8-9  7  1-2-3-4-5-6-7-8-9 | 1-2-3-4-5-6-7-8-9  1-2-3-4-5-6-7-8-9  1-2-3-4-5-6-7-8-9 
 6  1-2-3-4-5-6-7-8-9  1-2-3-4-5-6-7-8-9 | 1  9  5 | 1-2-3-4-5-6-7-8-9  1-2-3-4-5-6-7-8-9  1-2-3-4-5-6-7-8-9 
 1-2-3-4-5-6-7-8-9  9  8 | 1-2-3-4-5-6-7-8-9  1-2-3-4-5-6-7-8-9  1-2-3-4-5-6-7-8-9 | 1-2-3-4-5-6-7-8-9  6  1-2-3-4-5-6-7-8-9 
---------+---------+---------
 8  1-2-3-4-5-6-7-8-9  1-2-3-4-5-6-7-8-9 | 1-2-3-4-5-6-7-8-9  6  1-2-3-4-5-6-7-8-9 | 1-2-3-4-5-6-7-8-9  1-2-3-4-5-6-7-8-9  3 
 4  1-2-3-4-5-6-7-8-9  1-2-3-4-5-6-7-8-9 | 8  1-2-3-4-5-6-7-8-9  3 | 1-2-3-4-5-6-7-8-9  1-2-3-4-5-6-7-8-9  1 
 7  1-2-3-4-5-6-7-8-9  1-2-3-4-5-6-7-8-9 | 1-2-3-4-5-6-7-8-9  2  1-2-3-4-5-6-7-8-9 | 1-2-3-4-5-6-7-8-9  1-2-3-4-5-6-7-8-9  6 
---------+---------+---------
 1-2-3-4-5-6-7-8-9  6  1-2-3-4-5-6-7-8-9 | 1-2-3-4-5-6-7-8-9  1-2-3-4-5-6-7-8-9  1-2-3-4-5-6-7-8-9 | 2  8  1-2-3-4-5-6-7-8-9 
 1-2-3-4-5-6-7-8-9  1-2-3-4-5-6-7-8-9  1-2-3-4-5-6-7-8-9 | 4  1  9 | 1-2-3-4-5-6-7-8-9  1-

In [243]:
for block in sudoku.blocks(reduce=True):
    print(block)

Sudoku([SudokuEntry{5}, SudokuEntry{3},
        SudokuEntry{1, 2, 3, 4, 5, 6, 7, 8, 9}, SudokuEntry{6},
        SudokuEntry{1, 2, 3, 4, 5, 6, 7, 8, 9},
        SudokuEntry{1, 2, 3, 4, 5, 6, 7, 8, 9},
        SudokuEntry{1, 2, 3, 4, 5, 6, 7, 8, 9}, SudokuEntry{9},
        SudokuEntry{8}], dtype=object)
Sudoku([SudokuEntry{1, 2, 3, 4, 5, 6, 7, 8, 9}, SudokuEntry{7},
        SudokuEntry{1, 2, 3, 4, 5, 6, 7, 8, 9}, SudokuEntry{1},
        SudokuEntry{9}, SudokuEntry{5},
        SudokuEntry{1, 2, 3, 4, 5, 6, 7, 8, 9},
        SudokuEntry{1, 2, 3, 4, 5, 6, 7, 8, 9},
        SudokuEntry{1, 2, 3, 4, 5, 6, 7, 8, 9}], dtype=object)
Sudoku([SudokuEntry{1, 2, 3, 4, 5, 6, 7, 8, 9},
        SudokuEntry{1, 2, 3, 4, 5, 6, 7, 8, 9},
        SudokuEntry{1, 2, 3, 4, 5, 6, 7, 8, 9},
        SudokuEntry{1, 2, 3, 4, 5, 6, 7, 8, 9},
        SudokuEntry{1, 2, 3, 4, 5, 6, 7, 8, 9},
        SudokuEntry{1, 2, 3, 4, 5, 6, 7, 8, 9},
        SudokuEntry{1, 2, 3, 4, 5, 6, 7, 8, 9}, SudokuEntry{6},
        SudokuEntr

In [207]:
sudoku.get_row(1)

Sudoku([SudokuEntry{6}, SudokuEntry{1, 2, 3, 4, 5, 6, 7, 8, 9},
        SudokuEntry{1, 2, 3, 4, 5, 6, 7, 8, 9}, SudokuEntry{1},
        SudokuEntry{9}, SudokuEntry{5},
        SudokuEntry{1, 2, 3, 4, 5, 6, 7, 8, 9},
        SudokuEntry{1, 2, 3, 4, 5, 6, 7, 8, 9},
        SudokuEntry{1, 2, 3, 4, 5, 6, 7, 8, 9}], dtype=object)

In [196]:
type(sudoku.get(0, 0))

__main__.SudokuEntry

In [199]:
sudoku[0, :, 0, :].reshape(-1)

Sudoku([SudokuEntry{5}, SudokuEntry{3},
        SudokuEntry{1, 2, 3, 4, 5, 6, 7, 8, 9},
        SudokuEntry{1, 2, 3, 4, 5, 6, 7, 8, 9}, SudokuEntry{7},
        SudokuEntry{1, 2, 3, 4, 5, 6, 7, 8, 9},
        SudokuEntry{1, 2, 3, 4, 5, 6, 7, 8, 9},
        SudokuEntry{1, 2, 3, 4, 5, 6, 7, 8, 9},
        SudokuEntry{1, 2, 3, 4, 5, 6, 7, 8, 9}], dtype=object)

In [176]:
sudoku.get(1, 5)

5

In [204]:
for row in sudoku.rows():
    print(row)

Sudoku([[SudokuEntry{5}, SudokuEntry{3},
         SudokuEntry{1, 2, 3, 4, 5, 6, 7, 8, 9}],
        [SudokuEntry{1, 2, 3, 4, 5, 6, 7, 8, 9}, SudokuEntry{7},
         SudokuEntry{1, 2, 3, 4, 5, 6, 7, 8, 9}],
        [SudokuEntry{1, 2, 3, 4, 5, 6, 7, 8, 9},
         SudokuEntry{1, 2, 3, 4, 5, 6, 7, 8, 9},
         SudokuEntry{1, 2, 3, 4, 5, 6, 7, 8, 9}]], dtype=object)
Sudoku([[SudokuEntry{6}, SudokuEntry{1, 2, 3, 4, 5, 6, 7, 8, 9},
         SudokuEntry{1, 2, 3, 4, 5, 6, 7, 8, 9}],
        [SudokuEntry{1}, SudokuEntry{9}, SudokuEntry{5}],
        [SudokuEntry{1, 2, 3, 4, 5, 6, 7, 8, 9},
         SudokuEntry{1, 2, 3, 4, 5, 6, 7, 8, 9},
         SudokuEntry{1, 2, 3, 4, 5, 6, 7, 8, 9}]], dtype=object)
Sudoku([[SudokuEntry{1, 2, 3, 4, 5, 6, 7, 8, 9}, SudokuEntry{9},
         SudokuEntry{8}],
        [SudokuEntry{1, 2, 3, 4, 5, 6, 7, 8, 9},
         SudokuEntry{1, 2, 3, 4, 5, 6, 7, 8, 9},
         SudokuEntry{1, 2, 3, 4, 5, 6, 7, 8, 9}],
        [SudokuEntry{1, 2, 3, 4, 5, 6, 7, 8, 9}, SudokuE

In [216]:
print("\n\n".join("\n".join("x".join(str(k) for k in j) for j in i) for i in sudoku_grid.reshape(3,9,3).T.reshape(3,3,3,3)))

[ 0 27 54]x[ 3 30 57]x[ 6 33 60]
[ 9 36 63]x[12 39 66]x[15 42 69]
[18 45 72]x[21 48 75]x[24 51 78]

[ 1 28 55]x[ 4 31 58]x[ 7 34 61]
[10 37 64]x[13 40 67]x[16 43 70]
[19 46 73]x[22 49 76]x[25 52 79]

[ 2 29 56]x[ 5 32 59]x[ 8 35 62]
[11 38 65]x[14 41 68]x[17 44 71]
[20 47 74]x[23 50 77]x[26 53 80]


In [197]:
sudoku_grid = np.array(range(81)).reshape(3,3,3,3) 

In [198]:
print("\n\n".join("\n".join("x".join(str(k) for k in j) for j in i) for i in sudoku_grid.reshape(3,3,3,3)))

[0 1 2]x[3 4 5]x[6 7 8]
[ 9 10 11]x[12 13 14]x[15 16 17]
[18 19 20]x[21 22 23]x[24 25 26]

[27 28 29]x[30 31 32]x[33 34 35]
[36 37 38]x[39 40 41]x[42 43 44]
[45 46 47]x[48 49 50]x[51 52 53]

[54 55 56]x[57 58 59]x[60 61 62]
[63 64 65]x[66 67 68]x[69 70 71]
[72 73 74]x[75 76 77]x[78 79 80]


In [218]:
Sudoku(sudoku_grid)

 0  1  2 | 9  10  11 | 18  19  20 
 3  4  5 | 12  13  14 | 21  22  23 
 6  7  8 | 15  16  17 | 24  25  26 
---------+---------+---------
 27  28  29 | 36  37  38 | 45  46  47 
 30  31  32 | 39  40  41 | 48  49  50 
 33  34  35 | 42  43  44 | 51  52  53 
---------+---------+---------
 54  55  56 | 63  64  65 | 72  73  74 
 57  58  59 | 66  67  68 | 75  76  77 
 60  61  62 | 69  70  71 | 78  79  80 

In [7]:
# mat = """5 3 0 0 7 0 0 0 0
# 6 0 0 1 9 5 0 0 0
# 0 9 8 0 0 0 0 6 0
# 8 0 0 0 6 0 0 0 3
# 4 0 0 8 0 3 0 0 1
# 7 0 0 0 2 0 0 0 6
# 0 6 0 0 0 0 2 8 0 
# 0 0 0 4 1 9 0 0 5 
# 0 0 0 0 8 0 6 0 0"""

mat = """9 0 0 3 0 0 0 0 0
2 0 5 0 0 1 6 0 0
0 0 0 0 6 0 0 4 0
0 0 0 7 0 0 0 0 3
6 0 1 0 0 2 4 0 0
0 8 0 0 0 0 0 0 0
0 9 0 0 0 0 5 0 0
0 4 0 0 0 8 0 0 0
5 0 8 2 0 0 0 7 0"""

mat = [[*map(int, row.split())] for row in mat.split('\n')]

def matrix_to_sudoku(matrix, inner_rows):
    size = len(matrix)
    if size % inner_rows: 
        raise ValueError(f"Invalid value for inner_rows as {size} is not divisible by {inner_rows}")
    else: inner_cols = size // inner_rows
    block_rows = inner_cols
    block_cols = inner_rows
    sudoku_grid = [[[[matrix[block_row*inner_rows+inner_row][block_col*inner_cols+inner_col] for inner_col in range(inner_cols)] for inner_row in range(inner_rows)] 
                    for block_col in range(block_cols)] for block_row in range(block_rows)]
    return Sudoku(sudoku_grid)

print(matrix_to_sudoku(mat, 3))

 9  0  0 | 3  0  0 | 0  0  0 
 2  0  5 | 0  0  1 | 6  0  0 
 0  0  0 | 0  6  0 | 0  4  0 
---------+---------+---------
 0  0  0 | 7  0  0 | 0  0  3 
 6  0  1 | 0  0  2 | 4  0  0 
 0  8  0 | 0  0  0 | 0  0  0 
---------+---------+---------
 0  9  0 | 0  0  0 | 5  0  0 
 0  4  0 | 0  0  8 | 0  0  0 
 5  0  8 | 2  0  0 | 0  7  0 


In [145]:
class SudokuPuzzle:
    
    def __init__(self, input_array):
        problem = np.asarray(input_array)
        block_rows, block_cols, inner_rows, inner_cols = problem.shape
        self.problem = np.vectorize(lambda x: SudokuEntry(x))(problem)
        self.solution = self.get_solution()

    def get_solution(self):
        pass
    
    def __str__(self):
        for self

    
    

size = 9
inner_rows = 3
sudoku_grid = np.array([
    [[[5, 3, 0], [6, 0, 0], [0, 9, 8]],
     [[0, 7, 0], [1, 9, 5], [0, 0, 0]],
     [[0, 0, 0], [0, 0, 0], [0, 6, 0]]],
    [[[8, 0, 0], [4, 0, 0], [7, 0, 0]],
     [[0, 6, 0], [8, 0, 3], [0, 2, 0]],
     [[0, 0, 3], [0, 0, 1], [0, 0, 6]]],
    [[[0, 6, 0], [0, 0, 0], [0, 0, 0]],
     [[0, 0, 0], [4, 1, 9], [0, 8, 0]],
     [[2, 8, 0], [0, 0, 5], [6, 0, 0]]]
])
s = Sudoku(sudoku_grid)
print(s)

[SudokuEntry{5} SudokuEntry{3} SudokuEntry{1, 2, 3, 4, 5, 6, 7, 8, 9}]x[SudokuEntry{6} SudokuEntry{1, 2, 3, 4, 5, 6, 7, 8, 9}
 SudokuEntry{1, 2, 3, 4, 5, 6, 7, 8, 9}]x[SudokuEntry{1, 2, 3, 4, 5, 6, 7, 8, 9} SudokuEntry{9} SudokuEntry{8}]
[SudokuEntry{1, 2, 3, 4, 5, 6, 7, 8, 9} SudokuEntry{7}
 SudokuEntry{1, 2, 3, 4, 5, 6, 7, 8, 9}]x[SudokuEntry{1} SudokuEntry{9} SudokuEntry{5}]x[SudokuEntry{1, 2, 3, 4, 5, 6, 7, 8, 9}
 SudokuEntry{1, 2, 3, 4, 5, 6, 7, 8, 9}
 SudokuEntry{1, 2, 3, 4, 5, 6, 7, 8, 9}]
[SudokuEntry{1, 2, 3, 4, 5, 6, 7, 8, 9}
 SudokuEntry{1, 2, 3, 4, 5, 6, 7, 8, 9}
 SudokuEntry{1, 2, 3, 4, 5, 6, 7, 8, 9}]x[SudokuEntry{1, 2, 3, 4, 5, 6, 7, 8, 9}
 SudokuEntry{1, 2, 3, 4, 5, 6, 7, 8, 9}
 SudokuEntry{1, 2, 3, 4, 5, 6, 7, 8, 9}]x[SudokuEntry{1, 2, 3, 4, 5, 6, 7, 8, 9} SudokuEntry{6}
 SudokuEntry{1, 2, 3, 4, 5, 6, 7, 8, 9}]

[SudokuEntry{8} SudokuEntry{1, 2, 3, 4, 5, 6, 7, 8, 9}
 SudokuEntry{1, 2, 3, 4, 5, 6, 7, 8, 9}]x[SudokuEntry{4} SudokuEntry{1, 2, 3, 4, 5, 6, 7, 8, 9}
 SudokuE

TypeError: __str__ returned non-string (type NoneType)

# Sudoku Generator Algorithm:

In [375]:
sudoku_grid = np.array([
    [[[5, 3, 0], [6, 0, 0], [0, 9, 8]],
     [[0, 7, 0], [1, 9, 5], [0, 0, 0]],
     [[0, 0, 0], [0, 0, 0], [0, 6, 0]]],
    [[[8, 0, 0], [4, 0, 0], [7, 0, 0]],
     [[0, 6, 0], [8, 0, 3], [0, 2, 0]],
     [[0, 0, 3], [0, 0, 1], [0, 0, 6]]],
    [[[0, 6, 0], [0, 0, 0], [0, 0, 0]],
     [[0, 0, 0], [4, 1, 9], [0, 8, 0]],
     [[2, 8, 0], [0, 0, 5], [6, 0, 0]]]
])

sudoku = Sudoku(sudoku_grid)


# mat = """9 0 0 3 0 0 0 0 0
# 2 0 5 0 0 1 6 0 0
# 0 0 0 0 6 0 0 4 0
# 0 0 0 7 0 0 0 0 3
# 6 0 1 0 0 2 4 0 0
# 0 8 0 0 0 0 0 0 0
# 0 9 0 0 0 0 5 0 0
# 0 4 0 0 0 8 0 0 0
# 5 0 8 2 0 0 0 7 0"""

# mat = """0 7 0 0 0 1 0 0 5
# 0 4 0 5 2 0 0 6 0
# 0 0 2 0 0 3 0 0 0
# 0 0 0 1 0 0 0 0 0
# 0 0 8 4 5 0 0 0 7
# 7 0 0 0 0 0 0 3 0
# 0 0 0 0 0 2 0 0 0
# 0 0 4 7 8 0 0 0 3
# 0 6 0 0 0 0 9 0 0"""

# mat = """0 0 0 0 0 0 6 4 0
# 5 4 0 0 0 0 8 0 7
# 0 0 0 0 8 0 0 0 1
# 9 0 0 0 6 0 0 7 0
# 0 6 3 4 0 0 1 0 5
# 0 0 0 0 7 8 0 0 0
# 0 0 4 1 0 0 0 0 0
# 3 0 6 0 0 5 0 1 8
# 1 0 0 0 0 2 0 9 0"""

# mat = """9 0 0 0 0 2 0 0 7
# 0 0 0 0 0 7 0 5 0
# 0 0 0 0 6 0 0 1 3
# 0 0 0 6 0 0 0 0 9
# 0 9 0 0 3 0 1 0 2
# 7 0 0 2 1 0 0 0 0
# 0 0 1 0 0 0 0 4 0
# 2 0 0 0 0 0 8 0 1
# 0 0 0 0 5 0 0 0 0"""

# mat = """0 6 7 9 1 0 0 3 0
# 4 0 0 0 0 7 0 0 0
# 8 0 0 0 0 0 6 0 0
# 0 2 1 0 0 9 4 0 0
# 7 0 0 0 0 0 0 0 0
# 0 0 0 3 0 0 0 0 5
# 0 0 0 0 2 0 0 4 0
# 0 9 6 0 0 1 2 0 0
# 0 8 0 0 0 0 0 0 0"""

# mat = """0 9 0 7 0 1 0 0 0
# 0 0 0 4 0 0 0 0 0
# 7 0 0 0 0 6 0 0 0
# 0 1 0 0 0 0 0 0 4
# 0 0 0 0 9 5 0 0 7
# 6 0 8 0 4 0 0 9 0
# 8 0 0 3 0 0 7 0 0
# 0 0 4 0 5 0 0 0 2
# 0 2 9 0 0 0 0 5 8"""

# mat = """0 7 0 0 0 0 5 0 0
# 0 4 0 0 8 7 0 1 0
# 0 8 0 0 0 9 2 0 0
# 6 0 0 0 0 1 3 0 0
# 0 0 0 7 5 0 0 0 0
# 9 0 0 3 0 0 0 5 0
# 0 0 0 0 0 0 0 0 0
# 0 9 0 0 0 3 0 0 4
# 2 0 0 0 0 8 1 0 0"""

# mat = """0 0 1 0 0 4 0 8 0
# 0 0 5 0 0 0 0 0 7
# 4 7 0 0 3 0 0 6 0
# 6 4 0 0 0 8 0 0 9
# 1 0 0 0 0 0 0 0 0
# 0 0 0 0 5 0 8 0 0
# 0 0 0 2 0 0 0 3 0
# 0 0 7 0 0 0 0 0 0
# 8 9 0 0 0 5 0 0 6"""


# mat = [[*map(int, row.split())] for row in mat.split('\n')]
# sudoku = matrix_to_sudoku(mat, 3)

print("Before:")
print(sudoku)
print("Total Unknown:", sum((sudoku == 0).reshape(-1)))

def atleastTwoExists(a):
    I = np.ones(a.shape*3)
    AAI = a.reshape(-1, 1, 1) * a.reshape(1, -1, 1) * I
    AIA = a.reshape(-1, 1, 1) * I * a.reshape(1, 1, -1)
    IAA = I * a.reshape(1, -1, 1) * a.reshape(1, 1, -1)
    return AAI + AIA + IAA

def solve_sudoku(puzzle):
    sudoku = puzzle.copy()
    changes = 1
    while changes:
        changes = 0

        changes_difficult = 1
        while changes_difficult:
            changes_difficult = 0

            changes_medium = 1
            while changes_medium:
                changes_medium = 0

                changes_easy = 1
                while changes_easy:
                    changes_easy = 0
                    # Technique 1: Remove possibility of a certain entry value from all other entries in scope
                    # 
                    for val in range(1, np.multiply(*sudoku.shape[0:2])+1): # O(N)
                        for scope_gen in (sudoku.rows, sudoku.cols, sudoku.blocks):
                            for scope in scope_gen(reduce=True): # *O(N)
                                if val in scope: # *O(N)
                                    for entry in scope: # +O(N)
                                        if val in entry and entry != val:
                                            entry.remove_possibility(val) # *O(N)
                                            changes_easy += 1

                    # Technique 2: remove possibility of all other values from an entry only where a value is possible wrt any one scope
                    # 
                    for scope_gen in (sudoku.rows, sudoku.cols, sudoku.blocks):
                        for scope in scope_gen(reduce=True):
                            row = None
                            val_count = sum(entry.possible for entry in scope)
                            for val in (val_count == 1).nonzero()[0] + 1:
                                for entry in scope:
                                    if (val in entry) and (entry != val):
                                        entry.set_val(val)
                                        changes_easy += 1
                    changes += changes_easy



                # Technique 5: Block rays
                for  block_row, block_col, block in sudoku.blocks(enum=True):
                    null_pos = np.zeros(np.multiply(*block.shape))
                    pos_mat = np.asarray([[entry.possible if entry.count() > 1 else null_pos for entry in inner_row_entries] for inner_row_entries in block], dtype=bool)
                    for axis in range(2): #0:col, 1:row
                        pos_rays = pos_mat.sum(axis=axis, dtype=bool)
                        pos_weight = np.eye(len(pos_rays), dtype=np.int8) * 2 - 1
                        unique_rays = pos_weight.dot(pos_rays)
                        for inner_scope, val in zip(*(unique_rays>0).nonzero()):
                            val += 1
                            if axis: #row rays
                                for bcol in range(len(block)):
                                    if bcol != block_col:
                                        for entry in sudoku[block_row, bcol, inner_scope, :]:
                                            if val in entry and entry != val:
                                                entry.remove_possibility(val) # *O(N)
                                                changes_medium += 1

                            else: #col rays
                                for brow in range(len(block[0])):
                                    if brow != block_row:
                                        for entry in sudoku[brow, block_col, :, inner_scope]:
                                            if val in entry and entry != val:
                                                entry.remove_possibility(val) # *O(N)
                                                changes_medium += 1
    #             if changes_medium:
    #                 changes += changes_medium
    #                 continue

                # Technique 3: Hidden Pair by discovering exactly two values possible only in two entries
                # 
                for scope_gen in (sudoku.rows, sudoku.cols, sudoku.blocks):
                    for scope in scope_gen(reduce=True):
                        val_count = sum(entry.possible for entry in scope)
                        candidates = (val_count == 2).nonzero()[0]
                        if len(candidates) > 1:
                            val_val_count = sum(np.triu(entry.possible[candidates] * entry.possible[candidates].reshape(-1, 1), 1) for entry in scope)
                            for i1, i2 in zip(*(val_val_count == 2).nonzero()):
                                val1 = candidates[i1] + 1
                                val2 = candidates[i2] + 1
                                entry1, entry2 = [entry for entry in scope if val1 in entry]
                                if entry1.count() > 2:
                                    entry1.set_val([val1, val2])
                                    changes_medium += 1
                                if entry2.count() > 2:
                                    entry2.set_val([val1, val2])
                                    changes_medium += 1
                if changes_medium:
                    changes += changes_medium
                    continue

                # Technique 4: Other entries in scope of Hidden Pair entries must not carry hidden pair values
                # 
                for scope_gen in (sudoku.rows, sudoku.cols, sudoku.blocks):
                    for scope in scope_gen(reduce=True):
                        val_val_count = np.zeros((scope.shape[0], scope.shape[0])) + sum(np.triu(entry.possible * entry.possible.reshape(-1, 1), 1) for entry in scope if entry.count() == 2)
                        candidates = (val_val_count == 2).nonzero()
                        if len(candidates[0]):
                            hidden_pair_vals = np.concatenate(candidates)
                            for entry in scope:
                                new = entry.possible[hidden_pair_vals].sum()
                                if new and not ((new == 2) and (entry.count() == 2)):
                                    entry.possible[hidden_pair_vals] = False
                                    changes_medium += new
                if changes_medium:
                    changes += changes_medium
                    continue


            # Technique 6: Obvious Triplets:
            for scope_gen in (sudoku.rows, sudoku.cols, sudoku.blocks):
                for scope in scope_gen(reduce=True):
                    pos_scope = [entry.possible for entry in scope]
                    val_count = sum(pos_scope)
                    candidates = ((1 < val_count) & (val_count < 4)).nonzero()[0]
                    if len(candidates) > 2:
                        U = np.triu(np.ones(candidates.shape), 1)
                        UwU = U.reshape(*U.shape, 1) * U.reshape(1, *U.shape)
                        vvv_count = sum((atleastTwoExists(entry.possible[candidates]) * UwU).astype(bool) for entry in scope)
                        for i1, i2, i3 in zip(*(vvv_count >= 3).nonzero()):
                            val1 = candidates[i1] + 1
                            val2 = candidates[i2] + 1
                            val3 = candidates[i3] + 1
                            if len([entry for entry in scope if ((val1 in entry) or (val2 in entry))]) == 3:
                                for entry in scope:
                                    if (val1 in entry) or (val2 in entry):
                                        if entry.count() > 2:
                                            old_count = entry.count()
                                            indices = candidates[[i1, i2, i3]]
                                            old_pos = entry.possible[indices]
                                            entry.possible[:] = False
                                            entry.possible[indices] = old_pos
                                            changes_difficult += (old_count - entry.count())
                            else: continue

            changes += changes_difficult

        print(f"{changes = }")

    return sudoku

    
print()
solution = solve_sudoku(sudoku)
print("After:")
print(solution)
print("Total Unknown:", sum((sudoku == 0).reshape(-1)))

# sudoku

Before:
 5  3  0 | 0  7  0 | 0  0  0 
 6  0  0 | 1  9  5 | 0  0  0 
 0  9  8 | 0  0  0 | 0  6  0 
---------+---------+---------
 8  0  0 | 0  6  0 | 0  0  3 
 4  0  0 | 8  0  3 | 0  0  1 
 7  0  0 | 0  2  0 | 0  0  6 
---------+---------+---------
 0  6  0 | 0  0  0 | 2  8  0 
 0  0  0 | 4  1  9 | 0  0  5 
 0  0  0 | 0  8  0 | 6  0  0 
Total Unknown: 52

changes = 387
changes = 0
After:
 5  3  2 | 6  7  6 | 1  9  8 
 6  7  4 | 1  9  5 | 4  2  8 
 1  9  8 | 3  4  2 | 5  6  7 
---------+---------+---------
 8  5  9 | 7  6  1 | 4  2  3 
 4  2  6 | 8  5  3 | 9  7  1 
 7  1  3 | 9  2  4 | 8  5  6 
---------+---------+---------
 9  6  1 | 5  3  7 | 2  8  4 
 2  8  7 | 4  1  9 | 3  3  5 
 3  4  5 | 2  8  2 | 6  1  9 
Total Unknown: 52


# Create Sudoku Generator:

In [422]:
size = 12
block_rows = 3
if block_rows * block_rows <= size:
    block_cols = size // block_rows
    inner_rows = block_cols
    inner_cols = block_rows
    print(inner_rows, inner_cols)
    puzzle = np.zeros((block_rows, block_cols, inner_rows, inner_cols))
    for i in range(block_rows):
        puzzle[i, i, :, :] = np.random.permutation(np.arange(1, size+1)).reshape(inner_rows, inner_cols)
print(puzzle)
puzzle = Sudoku(puzzle)
print(puzzle)

4 3
[[[[ 6. 11. 10.]
   [ 7.  9. 12.]
   [ 5.  4.  3.]
   [ 2.  1.  8.]]

  [[ 0.  0.  0.]
   [ 0.  0.  0.]
   [ 0.  0.  0.]
   [ 0.  0.  0.]]

  [[ 0.  0.  0.]
   [ 0.  0.  0.]
   [ 0.  0.  0.]
   [ 0.  0.  0.]]

  [[ 0.  0.  0.]
   [ 0.  0.  0.]
   [ 0.  0.  0.]
   [ 0.  0.  0.]]]


 [[[ 0.  0.  0.]
   [ 0.  0.  0.]
   [ 0.  0.  0.]
   [ 0.  0.  0.]]

  [[ 2.  7. 10.]
   [ 6.  9.  3.]
   [ 1. 12.  4.]
   [11.  8.  5.]]

  [[ 0.  0.  0.]
   [ 0.  0.  0.]
   [ 0.  0.  0.]
   [ 0.  0.  0.]]

  [[ 0.  0.  0.]
   [ 0.  0.  0.]
   [ 0.  0.  0.]
   [ 0.  0.  0.]]]


 [[[ 0.  0.  0.]
   [ 0.  0.  0.]
   [ 0.  0.  0.]
   [ 0.  0.  0.]]

  [[ 0.  0.  0.]
   [ 0.  0.  0.]
   [ 0.  0.  0.]
   [ 0.  0.  0.]]

  [[11. 12.  6.]
   [ 9. 10.  4.]
   [ 7.  2.  3.]
   [ 5.  8.  1.]]

  [[ 0.  0.  0.]
   [ 0.  0.  0.]
   [ 0.  0.  0.]
   [ 0.  0.  0.]]]]


IndexError: arrays used as indices must be of integer (or boolean) type

In [386]:
np.random.shuffle

<function RandomState.shuffle>

In [326]:
# candidates[[0, 1]]

array([0, 2], dtype=int64)

In [392]:
print(sudoku)

 5  3  0 | 0  7  0 | 0  0  0 
 6  0  0 | 1  9  5 | 0  0  0 
 0  9  8 | 0  0  0 | 0  6  0 
---------+---------+---------
 8  0  0 | 0  6  0 | 0  0  3 
 4  0  0 | 8  0  3 | 0  0  1 
 7  0  0 | 0  2  0 | 0  0  6 
---------+---------+---------
 0  6  0 | 0  0  0 | 2  8  0 
 0  0  0 | 4  1  9 | 0  0  5 
 0  0  0 | 0  8  0 | 6  0  0 


In [396]:
print(sudoku.transpose(1,0,3,2))

 5  6  0 | 8  4  7 | 0  0  0 
 3  0  9 | 0  0  0 | 6  0  0 
 0  0  8 | 0  0  0 | 0  0  0 
---------+---------+---------
 0  1  0 | 0  8  0 | 0  4  0 
 7  9  0 | 6  0  2 | 0  1  8 
 0  5  0 | 0  3  0 | 0  9  0 
---------+---------+---------
 0  0  0 | 0  0  0 | 2  0  6 
 0  0  6 | 0  0  0 | 8  0  0 
 0  0  0 | 3  1  6 | 0  5  0 


In [38]:
16*16*16

4096

In [346]:
pos = entry.possible#.reshape(1, -1)

In [347]:
pos.T

array([ True, False, False,  True, False,  True, False, False,  True])

In [369]:
np.triu(np.multiply(pos, pos.reshape(-1, 1)), 1)

array([[False, False, False,  True, False,  True, False, False,  True],
       [False, False, False, False, False, False, False, False, False],
       [False, False, False, False, False, False, False, False, False],
       [False, False, False, False, False,  True, False, False,  True],
       [False, False, False, False, False, False, False, False, False],
       [False, False, False, False, False, False, False, False,  True],
       [False, False, False, False, False, False, False, False, False],
       [False, False, False, False, False, False, False, False, False],
       [False, False, False, False, False, False, False, False, False]])

In [360]:
np.multiply(pos, pos.reshape(-1, 1)).diag_indices()

AttributeError: 'numpy.ndarray' object has no attribute 'diag_indices'

In [357]:
np.vectorize(lambda x: x.possible * x.possible.reshape(1, -1))(row)

ValueError: setting an array element with a sequence.

In [368]:
np.triu(np.ones((9,9)), 1)

array([[0., 1., 1., 1., 1., 1., 1., 1., 1.],
       [0., 0., 1., 1., 1., 1., 1., 1., 1.],
       [0., 0., 0., 1., 1., 1., 1., 1., 1.],
       [0., 0., 0., 0., 1., 1., 1., 1., 1.],
       [0., 0., 0., 0., 0., 1., 1., 1., 1.],
       [0., 0., 0., 0., 0., 0., 1., 1., 1.],
       [0., 0., 0., 0., 0., 0., 0., 1., 1.],
       [0., 0., 0., 0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0.]])

In [447]:
entry

SudokuEntry{1, 2, 5, 6, 7, 9}

In [448]:
hidden_pair_vals

array([3, 7], dtype=int64)

In [451]:
entry.possible[hidden_pair_vals-1].sum()

1

In [116]:
a = np.asarray([1, 0, 0, 1, 1])

In [59]:
np.triu(a.reshape(-1, 1) * a.reshape(1, -1), 1) #.nonzero()

array([[0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 1],
       [0, 0, 0, 0, 0]])

In [189]:
a = np.asarray([1, 0, 1])
np.triu(np.triu(a.reshape(-1, 1) * a.reshape(1, -1), 1) * a.reshape(-1,1,1), 1).nonzero()

(array([0, 2], dtype=int64),
 array([0, 0], dtype=int64),
 array([2, 2], dtype=int64))

In [ ]:
#abc = ab + bc + ca

ab = np.triu(a.reshape(-1, 1) * a.reshape(1, -1), 1)


In [78]:
ab = np.triu(a.reshape(-1, 1) * a.reshape(1, -1), 1).reshape(-1, 1, 1)
bc = np.triu(a.reshape(-1, 1) * a.reshape(1, -1), 1).reshape(1, -1, 1)
ca = np.triu(a.reshape(-1, 1) * a.reshape(1, -1), 1).reshape(1, 1, -1)
# abc = 

ans = np.asarray(((ab + bc + ca) == 2).nonzero()).T
(ab + bc + ca).shape

(25, 25, 25)

In [214]:
(a | a.reshape(-1, 1))# * (a + a.reshape(-1, 1))

array([[0, 1, 0, 1],
       [1, 1, 1, 1],
       [0, 1, 0, 1],
       [1, 1, 1, 1]])

In [206]:
a = np.asarray([0, 1, 0, 1])
ab = np.triu(a.reshape(-1, 1) * a.reshape(1, -1), 1)
idxs = np.asarray((ab + a.reshape(-1, 1, 1)).nonzero())
idxs.T#[((idxs[0] < idxs[1]) & (idxs[1] < idxs[2]))]#.shape
ab
U = np.triu(np.ones(a.size), 1)
UwU = U.reshape(*U.shape, 1) * U.reshape(1, *U.shape)
UwU
(ab + a.reshape(-1, 1, 1))
(((ab + a.reshape(-1, 1, 1)) == 2) * UwU).astype(bool).nonzero()

(array([], dtype=int64), array([], dtype=int64), array([], dtype=int64))

In [201]:
ab

array([[0, 0, 0, 0],
       [0, 0, 0, 1],
       [0, 0, 0, 0],
       [0, 0, 0, 0]])

In [203]:
a.reshape(-1, 1, 1)

array([[[0]],

       [[1]],

       [[0]],

       [[1]]])

In [205]:
((ab + a.reshape(-1, 1, 1)) > 1)

array([[[False, False, False, False],
        [False, False, False, False],
        [False, False, False, False],
        [False, False, False, False]],

       [[False, False, False, False],
        [False, False, False,  True],
        [False, False, False, False],
        [False, False, False, False]],

       [[False, False, False, False],
        [False, False, False, False],
        [False, False, False, False],
        [False, False, False, False]],

       [[False, False, False, False],
        [False, False, False,  True],
        [False, False, False, False],
        [False, False, False, False]]])

In [137]:
np.triu(np.ones(a.size), 1).reshape(a.size, 1, a.size) * a.reshape(-1, 1, 1)

array([[[0., 1., 1., 1., 1.]],

       [[0., 0., 0., 0., 0.]],

       [[0., 0., 0., 0., 0.]],

       [[0., 0., 0., 0., 1.]],

       [[0., 0., 0., 0., 0.]]])

In [ ]:
0 0 0 0 0
0 0 1 1 1
0 0 0 1 1
0 0 0 0 1
0 0 0 0 0

0 0 0 0 0
0 0 0 0 0
0 0 0 1 1
0 0 0 0 1
0 0 0 0 0



0 1 1 1 1
0 0 1 1 1
0 0 0 1 1
0 0 0 0 1
0 0 0 0 0

array([[[0., 0., 0., 0., 0.],
        [0., 0., 1., 1., 1.],
        [0., 0., 0., 1., 1.],
        [0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0.]],

       [[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 1., 1.],
        [0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0.]],

       [[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0.]],

       [[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]],

       [[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]]])

In [148]:
sudoku.shape

(3, 3, 3, 3)

In [149]:
U = np.triu(np.ones(sudoku.shape[0] * sudoku.shape[1]), 1)
UwU = U.reshape(*U.shape, 1) * U.reshape(1, *U.shape)

In [207]:
UwU

array([[[0., 0., 0., 0.],
        [0., 0., 1., 1.],
        [0., 0., 0., 1.],
        [0., 0., 0., 0.]],

       [[0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 1.],
        [0., 0., 0., 0.]],

       [[0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.]],

       [[0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.]]])

In [155]:
candidates.shape

(4,)

In [156]:
UwU.shape

(12, 12, 12)

In [158]:
U = np.triu(np.ones(candidates.shape), 1)
# UwU = U.reshape(*U.shape, 1) * U.reshape(1, *U.shape)

In [159]:
U

array([[0., 1., 1., 1.],
       [0., 0., 1., 1.],
       [0., 0., 0., 1.],
       [0., 0., 0., 0.]])

In [230]:
aaa = a.reshape(-1, 1, 1)
(((aaa * aaa.transpose(1, 0, 2) * aaa.transpose(2, 1, 0)) - np.eye(a.size, dtype=int)[None, :, :])).nonzero()

(array([0, 0, 0, 0, 1, 1, 1, 1, 2, 2, 2, 2, 3, 3, 3, 3], dtype=int64),
 array([0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3], dtype=int64),
 array([0, 1, 2, 3, 0, 3, 2, 1, 0, 1, 2, 3, 0, 3, 2, 1], dtype=int64))

In [223]:
aaa

array([[[0]],

       [[1]],

       [[0]],

       [[1]]])

In [226]:
aaa.transpose(1, 0, 2)

array([[[0],
        [1],
        [0],
        [1]]])

In [241]:
def generate_b(a):
    size = len(a)
    I = np.eye(size)  # Identity matrix
    outer_product = np.einsum('i,j,k->ijk', a, a, a)  # Outer product of a with itself
#     b = outer_product - I  # Subtract identity to keep only cases where at least two of i, j, k exist
#     b = np.clip(b, 0, 1)  # Clip values between 0 and 1
    return outer_product

In [242]:
generate_b(a).nonzero()

(array([1, 1, 1, 1, 3, 3, 3, 3], dtype=int64),
 array([1, 1, 3, 3, 1, 1, 3, 3], dtype=int64),
 array([1, 3, 1, 3, 1, 3, 1, 3], dtype=int64))

# .
# .
# .























In [315]:
a = np.asarray([1, 0, 1, 1])

In [316]:
i = np.asarray([0, 1, 3])

In [319]:
store = a[i] & 1
a[:] = 0
a[i] = store

In [320]:
a

array([1, 0, 0, 1])

In [263]:
a * a.reshape(-1, 1)

array([[1, 0, 0],
       [0, 0, 0],
       [0, 0, 0]])

In [260]:
U = np.triu(np.ones(a.size), 1)
UwU = U.reshape(*U.shape, 1) * U.reshape(1, *U.shape)
UwU

array([[[0., 0., 0.],
        [0., 0., 1.],
        [0., 0., 0.]],

       [[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]],

       [[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]]])

In [281]:
((a * a.reshape(-1, 1)) + a.reshape(-1, 1, 1))

# 1 1 1
# 1 0 0
# 1 0 0

# 1 0 0
# 0 0 0
# 0 0 0

# 1 0 0
# 0 0 0
# 0 0 0


# (a * a.reshape(-1, 1)) * np.ones(shape=(*a.shape, 1, 1))
a * np.ones(shape=(*a.shape, 1, 1)) #* a.reshape(-1, 1)

array([[[1., 0., 0.]],

       [[1., 0., 0.]],

       [[1., 0., 0.]]])

In [282]:
((a * a.reshape(-1, 1)) + a.reshape(-1, 1, 1))

array([[[2, 1, 1],
        [1, 1, 1],
        [1, 1, 1]],

       [[1, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[1, 0, 0],
        [0, 0, 0],
        [0, 0, 0]]])

In [ ]:
0 1 3, 1 2 3

0 0 0 0
0 0 0 1
0 0 0 0
0 1 0 0

0 0 0 1
0 0 0 0
1 0 0 1
0 0 1 0

0 0 0 0
0 0 0 1
0 0 0 0
0 1 0 0

0 1 0 0
1 0 1 0
0 1 0 0
0 0 0 0

In [297]:
((a * a.reshape(-1, 1)) + a.reshape(-1, 1, 1))

# 1 1 1
# 1 0 0
# 1 0 0

# 1 0 0
# 0 0 0
# 0 0 0

# 1 0 0
# 0 0 0
# 0 0 0


(a * a.reshape(-1, 1)) * np.ones((*a.shape, 1, 1))

AAI = a.reshape(-1, 1, 1) * a.reshape(1, -1, 1) * np.ones(a.shape*3)
AIA = a.reshape(-1, 1, 1) * np.ones(a.shape*3) * a.reshape(1, 1, -1)
IAA = np.ones(a.shape*3) * a.reshape(1, -1, 1) * a.reshape(1, 1, -1)
AAI + AIA + IAA
# EA = ((a * np.ones((*a.shape, 1)) + np.ones(a.shape) * a.reshape(-1,1)))
# EA * a.reshape(*a.shape, 1, 1)
# EA

array([[[3., 1., 1.],
        [1., 0., 0.],
        [1., 0., 0.]],

       [[1., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]],

       [[1., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]]])

In [300]:
a

array([1, 0, 0])

In [248]:
a.shape = (4, 1, 1)

In [250]:
a.shape = 4

In [251]:
a

array([0, 1, 0, 1])

array([1, 0, 0, 1])